В этом ноутбуке:
- предобработка данных

# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../scripts/')

import substat

# Случайные замены

Загрузим данные:

In [2]:
main_df = pd.read_csv("../data/Random.main.tsv.gz", sep="\t", low_memory=False)
print(main_df.shape)
main_df.head()

(190488, 36)


,#PDBId,PDBChain,PDBRes,AA1,AA2,Blosum62,Grantham,FoldXddG,MaestroddG,MaestroConf,...,Hb.SM,Hb.SS,IntCont,AveSeqDist,ExtCont,ExtList,BiolHetCont,BiolHetList,NonBiolHetCont,NonBiolHetList
0,6icz,A,1261,N,T,0,65,0.902437,-2.159708,0.221499,...,0,0,8,2.5,0,?,0,?,0,?
1,6icz,A,991,T,K,-1,78,-1.729310,-2.044080,0.256225,...,0,0,14,30.0,12,"L,R",0,?,0,?
2,4ksl,A,102,K,I,-3,102,0.460387,-0.411635,0.780515,...,0,0,16,31.6,4,D,0,?,0,?
3,6icz,A,1797,N,S,1,46,-0.162796,-2.183104,0.199743,...,0,0,7,3.0,3,"F,G",0,?,0,?
4,6id0,A,1449,K,I,-3,102,-0.438241,-1.565762,0.399666,...,0,0,6,131.7,0,?,0,?,0,?


Уберём ненужные колонки:

In [3]:
main_df.columns

Index(['#PDBId', 'PDBChain', 'PDBRes', 'AA1', 'AA2', 'Blosum62', 'Grantham',
       'FoldXddG', 'MaestroddG', 'MaestroConf', 'SimpleClassRIN', 'dVol',
       'dProp', 'DSSP.NormASA', 'DSSP.SecStr', 'DSSP.Phi', 'DSSP.Psi',
       'MapRegion', 'NumAtoms', 'B.Mean', 'B.RMSD', 'B.Zscore', 'Hb.Tot',
       'Hb.Inter', 'Hb.Intra', 'Hb.IntraDist', 'Hb.SM', 'Hb.SS', 'IntCont',
       'AveSeqDist', 'ExtCont', 'ExtList', 'BiolHetCont', 'BiolHetList',
       'NonBiolHetCont', 'NonBiolHetList'],
      dtype='object')

In [4]:
main_df = main_df[['AA1', 'AA2', 'Blosum62', 'Grantham',
                   'FoldXddG', 'MaestroddG', 'SimpleClassRIN', 'dVol',
                   'dProp', 'DSSP.NormASA', 'DSSP.SecStr', 'B.Mean', 'B.Zscore', 
                   'Hb.Tot', 'Hb.Inter', 'Hb.Intra', 'Hb.IntraDist', 'Hb.SM', 'Hb.SS', 
                   'IntCont', 'AveSeqDist', 'ExtCont', 'BiolHetCont']]
print(main_df.shape)
main_df.head()

(190488, 23)


,AA1,AA2,Blosum62,Grantham,FoldXddG,MaestroddG,SimpleClassRIN,dVol,dProp,DSSP.NormASA,...,Hb.Tot,Hb.Inter,Hb.Intra,Hb.IntraDist,Hb.SM,Hb.SS,IntCont,AveSeqDist,ExtCont,BiolHetCont
0,N,T,0,65,0.902437,-2.159708,Protein interaction,2,-0.39,0.481,...,0,0,0,0,0,0,8,2.5,0,0
1,T,K,-1,78,-1.729310,-2.044080,Protein interaction,52,2.1,0.0504,...,0,0,0,0,0,0,14,30.0,12,0
2,K,I,-3,102,0.460387,-0.411635,Protein interaction,-1,-2.8,0.264,...,0,0,0,0,0,0,16,31.6,4,0
3,N,S,1,46,-0.162796,-2.183104,RNA interaction,-25,-0.34,0.769,...,0,0,0,0,0,0,7,3.0,3,0
4,K,I,-3,102,-0.438241,-1.565762,Protein interaction,-1,-2.8,0.523,...,0,0,0,0,0,0,6,131.7,0,0


Добавим гидрофобность для остатка дикого типа и изменение гидрофобности после замены (займёт некоторое время):

In [7]:
main_df['ResKDHydro'] = main_df.apply(lambda x: 
                                          substat.calc_hydro('../data/hydrophobicity.tsv', 'kD', x['AA1']), 
                                          axis=1)
main_df.head()

,AA1,AA2,Blosum62,Grantham,FoldXddG,MaestroddG,SimpleClassRIN,dVol,dProp,DSSP.NormASA,...,Hb.Inter,Hb.Intra,Hb.IntraDist,Hb.SM,Hb.SS,IntCont,AveSeqDist,ExtCont,BiolHetCont,ResKDHydro
0,N,T,0,65,0.902437,-2.159708,Protein interaction,2,-0.39,0.481,...,0,0,0,0,0,8,2.5,0,0,-3.5
1,T,K,-1,78,-1.729310,-2.044080,Protein interaction,52,2.1,0.0504,...,0,0,0,0,0,14,30.0,12,0,-0.7
2,K,I,-3,102,0.460387,-0.411635,Protein interaction,-1,-2.8,0.264,...,0,0,0,0,0,16,31.6,4,0,-3.9
3,N,S,1,46,-0.162796,-2.183104,RNA interaction,-25,-0.34,0.769,...,0,0,0,0,0,7,3.0,3,0,-3.5
4,K,I,-3,102,-0.438241,-1.565762,Protein interaction,-1,-2.8,0.523,...,0,0,0,0,0,6,131.7,0,0,-3.9


In [8]:
main_df['SubDiffKDHydro'] = main_df.apply(lambda x: 
                                          substat.diff_hydro('../data/hydrophobicity.tsv', 'kD', x['AA1'], x['AA2']), 
                                          axis=1)
main_df.head()

,AA1,AA2,Blosum62,Grantham,FoldXddG,MaestroddG,SimpleClassRIN,dVol,dProp,DSSP.NormASA,...,Hb.Intra,Hb.IntraDist,Hb.SM,Hb.SS,IntCont,AveSeqDist,ExtCont,BiolHetCont,ResKDHydro,SubDiffKDHydro
0,N,T,0,65,0.902437,-2.159708,Protein interaction,2,-0.39,0.481,...,0,0,0,0,8,2.5,0,0,-3.5,2.8
1,T,K,-1,78,-1.729310,-2.044080,Protein interaction,52,2.1,0.0504,...,0,0,0,0,14,30.0,12,0,-0.7,-3.2
2,K,I,-3,102,0.460387,-0.411635,Protein interaction,-1,-2.8,0.264,...,0,0,0,0,16,31.6,4,0,-3.9,8.4
3,N,S,1,46,-0.162796,-2.183104,RNA interaction,-25,-0.34,0.769,...,0,0,0,0,7,3.0,3,0,-3.5,2.7
4,K,I,-3,102,-0.438241,-1.565762,Protein interaction,-1,-2.8,0.523,...,0,0,0,0,6,131.7,0,0,-3.9,8.4


Сохраним новую таблицу:

In [9]:
main_df.to_csv('../results/random_preprocessed.csv.gz', index=False, compression="gzip")

# Замены ClinVar

Загрузим данные:

In [2]:
main_df = pd.read_csv("../data/ClinVar.191020.main.tsv.bz2", sep="\t", low_memory=False)
print(main_df.shape)
main_df.head()

(216255, 103)


,#ID,Acc,Pos,AA1,AA2,Gene,PPH.PtGgPaNl,PhyloP,rsID,PPH.Pred,...,CV.Phenotype,CV.Origin,CV.Review,oe_mis,oe_lof,NumAA2in26Prim,ClinSig.200914,AA2Carrier117,MaxIdeAA2Carrier117,NumAA2in117
0,chr1:1014012_C/T,P05161,11,A,V,ISG15,C>C>C>?,-2.569,rs950572063,benign,...,Immunodeficiency 38 with basal ganglia calcifi...,germline,reliable,1.0115e+00,0.0000e+00,0,VUS,colLiv1,0.373,3
1,chr1:1014042_G/A,P05161,21,S,N,ISG15,A>A>A>?,-0.676,rs143888043,benign,...,Immunodeficiency 38 with basal ganglia calcifi...,germline,reliable,1.0115e+00,0.0000e+00,20,Benign,panPan2,0.988,34
2,chr1:1014047_A/G,P05161,23,M,V,ISG15,A>A>A>?,0.592,rs374095998,possibly damaging,...,Immunodeficiency 38 with basal ganglia calcifi...,germline,reliable,1.0115e+00,0.0000e+00,0,VUS,danRer10,0.358,3
3,chr1:1014111_G/A,P05161,44,R,H,ISG15,G>G>G>?,0.126,rs148041041,benign,...,Immunodeficiency 38 with basal ganglia calcifi...,germline,reliable,1.0115e+00,0.0000e+00,3,VUS,otoGar3,0.721,5
4,chr1:1014122_C/T,P05161,48,H,Y,ISG15,C>C>C>?,0.081,rs150861311,benign,...,Immunodeficiency 38 with basal ganglia calcifi...,germline,reliable,1.0115e+00,0.0000e+00,1,VUS,calJac3,0.825,2


Уберём ненужные колонки:

In [15]:
main_df = main_df[['AA1', 'AA2', 'PhyloP', 'Blosum62', 'Grantham', 
                   'PPH.dScore', 'PPH.Site', 'PPH.Region', 'PPH.PHAT',
                   'SPOTD.Info', 'SPOTD.PDisorder', 'SPOTD.PCoil', 'SPOTD.PSheet', 'SPOTD.PHelix', 
                   'SPOTD.CN', 'SPOTD.ASA', 'SPOTD.HSEu', 'SPOTD.HSEd', 'SPOTD.SecStr', 
                   'SM.ClassRIN', 'SM.Ide', 'SM.Coverage', 'SM.Resol', 'PDBAAMatch',
                   'MaestroddG', 'FoldXddG', 'dVol', 'dProp', 
                   'DSSP.NormASA', 'DSSP.SecStr',
                   'B.Mean', 'B.Zscore', 
                   'Hb.Tot', 'Hb.Inter', 'Hb.Intra', 'Hb.IntraDist', 'Hb.SM', 'Hb.SS',
                   'IntCont', 'AveSeqDist', 'ExtCont', 'BiolHetCont', 
                   'NumAA2in26Prim', 'NumAA2in117', 'CV.SigShort']]
main_df.rename(columns={'SM.ClassRIN': 'SimpleClassRIN'})

print(main_df.shape)
main_df.head()

(216255, 45)


,AA1,AA2,PhyloP,Blosum62,Grantham,PPH.dScore,PPH.Site,PPH.Region,PPH.PHAT,SPOTD.Info,...,Hb.IntraDist,Hb.SM,Hb.SS,IntCont,AveSeqDist,ExtCont,BiolHetCont,NumAA2in26Prim,NumAA2in117,CV.SigShort
0,A,V,-2.569,0,64,+1.483,NO,NO,?,0.47,...,0,0,0,4,1.5,1,0,0,3,VUS
1,S,N,-0.676,1,46,-0.430,NO,NO,?,0.53,...,0,0,0,4,1.5,0,0,20,34,Benign
2,M,V,0.592,1,21,+1.539,NO,NO,?,0.70,...,0,0,0,12,21.1,0,0,0,3,VUS
3,R,H,0.126,0,29,+0.816,NO,NO,?,1.01,...,0,2,0,11,15.1,0,0,3,5,VUS
4,H,Y,0.081,2,83,+0.877,NO,NO,?,0.53,...,0,0,0,9,13.1,2,0,1,2,VUS


Добавим гидрофобность для остатка дикого типа и изменение гидрофобности после замены (займёт некоторое время):

In [16]:
main_df['ResKDHydro'] = main_df.apply(lambda x: 
                                          substat.calc_hydro('../data/hydrophobicity.tsv', 'kD', x['AA1']), 
                                          axis=1)
main_df.head()

<ipython-input-16-01032e0937c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['ResKDHydro'] = main_df.apply(lambda x:


,AA1,AA2,PhyloP,Blosum62,Grantham,PPH.dScore,PPH.Site,PPH.Region,PPH.PHAT,SPOTD.Info,...,Hb.SM,Hb.SS,IntCont,AveSeqDist,ExtCont,BiolHetCont,NumAA2in26Prim,NumAA2in117,CV.SigShort,ResKDHydro
0,A,V,-2.569,0,64,+1.483,NO,NO,?,0.47,...,0,0,4,1.5,1,0,0,3,VUS,1.8
1,S,N,-0.676,1,46,-0.430,NO,NO,?,0.53,...,0,0,4,1.5,0,0,20,34,Benign,-0.8
2,M,V,0.592,1,21,+1.539,NO,NO,?,0.70,...,0,0,12,21.1,0,0,0,3,VUS,1.9
3,R,H,0.126,0,29,+0.816,NO,NO,?,1.01,...,2,0,11,15.1,0,0,3,5,VUS,-4.5
4,H,Y,0.081,2,83,+0.877,NO,NO,?,0.53,...,0,0,9,13.1,2,0,1,2,VUS,-3.2


In [17]:
main_df['SubDiffKDHydro'] = main_df.apply(lambda x: 
                                          substat.diff_hydro('../data/hydrophobicity.tsv', 'kD', x['AA1'], x['AA2']), 
                                          axis=1)
main_df.head()

<ipython-input-17-cee20dc9ef28>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df['SubDiffKDHydro'] = main_df.apply(lambda x:


,AA1,AA2,PhyloP,Blosum62,Grantham,PPH.dScore,PPH.Site,PPH.Region,PPH.PHAT,SPOTD.Info,...,Hb.SS,IntCont,AveSeqDist,ExtCont,BiolHetCont,NumAA2in26Prim,NumAA2in117,CV.SigShort,ResKDHydro,SubDiffKDHydro
0,A,V,-2.569,0,64,+1.483,NO,NO,?,0.47,...,0,4,1.5,1,0,0,3,VUS,1.8,2.4
1,S,N,-0.676,1,46,-0.430,NO,NO,?,0.53,...,0,4,1.5,0,0,20,34,Benign,-0.8,-2.7
2,M,V,0.592,1,21,+1.539,NO,NO,?,0.70,...,0,12,21.1,0,0,0,3,VUS,1.9,2.3
3,R,H,0.126,0,29,+0.816,NO,NO,?,1.01,...,0,11,15.1,0,0,3,5,VUS,-4.5,1.3
4,H,Y,0.081,2,83,+0.877,NO,NO,?,0.53,...,0,9,13.1,2,0,1,2,VUS,-3.2,1.9


Сохраним новую таблицу:

In [18]:
main_df.to_csv('../results/ClinVar_preprocessed.csv.gz', index=False, compression="gzip")